In [1]:
!pip install wandb

In [2]:
! pip install pytorch_lightning

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
import wandb
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from kaggle_secrets import UserSecretsClient



In [3]:
# Simple CNN Model with flexibile  conv blocks, each: Conv -> (optional BatchNorm) -> Activation -> MaxPool
class SimpleCNN(pl.LightningModule):
    def __init__(self, conv_filters=[32, 32, 32, 32, 32],
                 conv_kernel_sizes=[3, 3, 3, 3, 3],
                 conv_activation='ReLU',
                 dense_neurons=128,
                 dense_activation='ReLU',
                 dropout=0.0,
                 use_batchnorm=False,
                 lr=1e-3):
        super(SimpleCNN, self).__init__()
        self.save_hyperparameters()
        # Dictionary mapping activation names to modules
        activations = {
            'ReLU': nn.ReLU,
            'GELU': nn.GELU,
            'SiLU': nn.SiLU
            
        }
        conv_act = activations.get(conv_activation, nn.ReLU)
        dense_act = activations.get(dense_activation, nn.ReLU)
        
        self.conv_layers = nn.ModuleList()
        in_channels = 3  # iNaturalist images have 3 channels (RGB)
        for i in range(5):
            out_channels = conv_filters[i]
            kernel_size = conv_kernel_sizes[i]
            conv = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=kernel_size//2)
            block = [conv]
            if use_batchnorm:
                block.append(nn.BatchNorm2d(out_channels))
            block.append(conv_act())
            block.append(nn.MaxPool2d(2))
            self.conv_layers.append(nn.Sequential(*block))
            in_channels = out_channels
        
        # Compute the flattened feature size after conv layers
        # Assume a default input image size of 224x224
        dummy_input = torch.zeros(1, 3, 224, 224)
        with torch.no_grad():
            features = dummy_input
            for layer in self.conv_layers:
                features = layer(features)
            self.flattened_size = features.view(1, -1).shape[1]
        
        # Dense layer followed by the output layer (10 neurons for 10 classes)
        self.dense = nn.Linear(self.flattened_size, dense_neurons)
        self.dense_activation = dense_act()
        self.dropout = nn.Dropout(dropout) if dropout > 0 else None
        self.out = nn.Linear(dense_neurons, 10)
        self.lr = lr

    def forward(self, x):
        for layer in self.conv_layers:
            x = layer(x)
        x = torch.flatten(x, 1)
        x = self.dense(x)
        x = self.dense_activation(x)
        if self.dropout:
            x = self.dropout(x)
        x = self.out(x)
        return x
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        acc = (logits.argmax(dim=1) == y).float().mean()
        self.log('train_loss', loss)
        self.log('train_acc', acc)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        acc = (logits.argmax(dim=1) == y).float().mean()
        self.log('val_loss', loss, prog_bar=True)
        self.log('val_acc', acc, prog_bar=True)
    
    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        acc = (logits.argmax(dim=1) == y).float().mean()
        self.log('test_loss', loss)
        self.log('test_acc', acc)
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        return optimizer

In [2]:
class INaturalistDataModule(pl.LightningDataModule):
    def __init__(self, data_dir, batch_size=32, num_workers=4, valid_split=0.2, use_augmentation=True):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.valid_split = valid_split
        self.use_augmentation = use_augmentation  # Enable/Disable augmentation

        # Mean and std from ImageNet (often used for natural image datasets)
        self.mean = [0.485, 0.456, 0.406]
        self.std = [0.229, 0.224, 0.225]

        # **Train Transform with Data Augmentation**
        if self.use_augmentation:
            self.train_transform = transforms.Compose([
                transforms.Resize((224, 224)),  
                transforms.RandomHorizontalFlip(),  # Flip images randomly
                transforms.RandomRotation(10),  # Rotate within ±10 degrees
                transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Adjust colors
                transforms.ToTensor(),
                transforms.Normalize(self.mean, self.std)  # Normalize
            ])
        else:
            self.train_transform = transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize(self.mean, self.std)
            ])

        # **Validation & Test Transform (No Augmentation)**
        self.test_transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(self.mean, self.std)
        ])
    
    def setup(self, stage=None):
        # Load dataset
        full_train_dataset = torchvision.datasets.ImageFolder(root=f'{self.data_dir}/train', transform=self.train_transform)
        self.test_dataset = torchvision.datasets.ImageFolder(root=f'{self.data_dir}/val', transform=self.test_transform)

        # **Split train dataset into train (80%) & validation (20%)**
        total = len(full_train_dataset)
        val_size = int(total * self.valid_split)
        train_size = total - val_size
        self.train_dataset, self.val_dataset = random_split(full_train_dataset, [train_size, val_size])

        # **Set validation transform separately**
        self.val_dataset.dataset.transform = self.test_transform
        return self.train_dataset,self.val_dataset,self.test_dataset

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=self.num_workers)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, shuffle=False, num_workers=self.num_workers)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=False, num_workers=self.num_workers)


2000

In [3]:
# Sweep configuration for wandb hyperparameter search
sweep_config = {
    'method': 'bayes',
    'metric': {
      'name': 'val_acc',
      'goal': 'maximize'   
    },
    'parameters': {
        'conv_filters': {
            'values': [[32, 32, 32, 32, 32], [32, 64, 128, 256, 512]]
        },
        'conv_kernel_sizes': {
            'values': [[3, 3, 3, 3, 3], [5, 3, 3, 3, 3]]
        },
        'conv_activation': {
            'values': ['ReLU', 'SiLU', 'GeLU']
        },
        'dense_neurons': {
            'values': [128, 256]
        },
        'dense_activation': {
            'values': ['ReLU', 'SiLU']
        },
        'dropout': {
            'values': [0.2, 0.3]
        },
        'use_batchnorm': {
            'values': [True, False]
        },
        'lr': {
            'values': [1e-3, 1e-4]
        },
        'batch_size': {
            'values': [32, 64]
        },
        'use_augmentation': {  
            'values': [True, False]
        }
    }
}

In [ ]:
def train():
    # Initialize wandb run and get hyperparameters
    wandb.init(project="Simple_cnn")  # Add project name
    config = wandb.config

    # Debug: Print config to verify keys exist
    print("Loaded Config:", config)

    
    model = SimpleCNN(
        conv_filters=config.get("conv_filters", [32, 32, 32, 32, 32]),  # Default filters
        conv_kernel_sizes=config.get("conv_kernel_sizes", [3, 3, 3, 3, 3]),
        conv_activation=config.get("conv_activation", "ReLU"),
        dense_neurons=config.get("dense_neurons", 128),
        dense_activation=config.get("dense_activation", "ReLU"),
        dropout=config.get("dropout", 0.2),
        use_batchnorm=config.get("use_batchnorm", False),
        lr=config.get("lr", 1e-3)
    )

    data_module = INaturalistDataModule(
        data_dir='/kaggle/input/inaturalist-dataset/inaturalist_12K',
        batch_size=config.get("batch_size", 32),
        use_augmentation=config.get("use_augmentation", False) 
    )
    data_module.setup()
    train_loader = data_module.train_dataloader()
    val_loader = data_module.val_dataloader()
    test_loader = data_module.test_dataloader()

    # Log training process with wandb
    wandb_logger = WandbLogger(project="Simple_cnn", log_model="all")

    # Define Trainer pytorch_lighting method which automatically trains model and logs metrics 
    trainer = pl.Trainer(
        max_epochs=10,   
        logger=wandb_logger,
    )

    # Train the model
    trainer.fit(model, train_loader,val_loader)
 
if __name__ == "__main__":
    
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    secret_value_0 = user_secrets.get_secret("wandb")
    wandb.login(key=secret_value_0)
    sweep_id = wandb.sweep(sweep_config,project='Simple_cnn')
    # Run multiple experiments using a sweep
    wandb.agent(sweep_id, train, count=30)  # Runs 30 experiments


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: ll2zrcl4
Sweep URL: https://wandb.ai/da24m026-indian-institute-of-technology-madras/Simple_cnn/sweeps/ll2zrcl4


wandb: Agent Starting Run: 5tztsja0 with config:
wandb: 	batch_size: 32
wandb: 	conv_activation: GeLU
wandb: 	conv_filters: [32, 64, 128, 256, 512]
wandb: 	conv_kernel_sizes: [3, 3, 3, 3, 3]
wandb: 	dense_activation: SiLU
wandb: 	dense_neurons: 128
wandb: 	dropout: 0.2
wandb: 	lr: 0.0001
wandb: 	use_augmentation: True
wandb: 	use_batchnorm: False
wandb: WARNING Ignoring project 'Simple_cnn' when running a sweep.


Loaded Config: {'batch_size': 32, 'conv_activation': 'GeLU', 'conv_filters': [32, 64, 128, 256, 512], 'conv_kernel_sizes': [3, 3, 3, 3, 3], 'dense_activation': 'SiLU', 'dense_neurons': 128, 'dropout': 0.2, 'lr': 0.0001, 'use_augmentation': True, 'use_batchnorm': False}


/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loggers/wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
wandb: WARNING Config item 'conv_filters' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'conv_kernel_sizes' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'conv_activation' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'dense_neurons' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'dense_activation' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'dropout' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'use_batchnorm' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'lr' was locked by 'sweep' (ignored update).


...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

# Part A Question:4

In [ ]:
# best model from training :'batch_size': 64, 'conv_activation': 'GeLU', 'conv_filters': [32, 64, 128, 256, 512], 'conv_kernel_sizes': [3, 3, 3, 3, 3], 'dense_activation': 'SiLU', 'dense_neurons': 256, 'dropout': 0.3, 'lr': 0.0001, 'use_augmentation': False,

In [8]:

import torch
import wandb
import pytorch_lightning as pl
import matplotlib.pyplot as plt
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger
from kaggle_secrets import UserSecretsClient
from torchvision.utils import make_grid

# load WandB API Key from Kaggle Secrets
user_secrets = UserSecretsClient()
wandb_key = user_secrets.get_secret("wandb")
wandb.login(key=wandb_key)

#  Initialize WandB Project
wandb.init(project="Simple_cnn", name="best_model_eval")

#  Load the best model from sweeps
best_model = SimpleCNN(
    conv_filters=[32, 64, 128, 256, 512], 
    conv_kernel_sizes=[3, 3, 3, 3, 3],
    conv_activation="GeLU",
    dense_neurons=256,
    dense_activation="SiLU",
    dropout=0.3,
    use_batchnorm=True,
    lr=0.0001
)

#  Load dataset
data_module = INaturalistDataModule(
    data_dir='/kaggle/input/inaturalist-dataset/inaturalist_12K',
    batch_size=64,
    use_augmentation=False
)
data_module.setup()

test_loader = data_module.test_dataloader()

# Load WandB Logger
wandb_logger = WandbLogger(project="Simple_cnn", log_model="all")


trainer = pl.Trainer(
    max_epochs=10,
    logger=wandb_logger,
)

trainer.fit(best_model, data_module)

test_results = trainer.test(best_model, test_loader)
test_accuracy = test_results[0]['test_acc']


wandb.log({"Test Accuracy": test_accuracy})



def log_test_predictions(model, dataloader, num_classes=10, num_per_class=3):
    model.eval()
    class_images = {i: [] for i in range(num_classes)}  # Store images per class

    with torch.no_grad():
        for images, labels in dataloader:
            outputs = model(images)
            preds = torch.argmax(outputs, dim=1)

            # Collect 3 images per class
            for img, label, pred in zip(images, labels, preds):
                if len(class_images[label.item()]) < num_per_class:
                    class_images[label.item()].append((img, pred.item()))

            # Stop if we have enough samples
            if all(len(class_images[i]) == num_per_class for i in range(num_classes)):
                break

    # Create a grid
    fig, axes = plt.subplots(num_classes, num_per_class, figsize=(num_per_class * 3, num_classes * 3))
    
    for class_idx, ax_row in enumerate(axes):
        for img_idx, ax in enumerate(ax_row):
            if class_idx in class_images and len(class_images[class_idx]) > img_idx:
                img, pred_label = class_images[class_idx][img_idx]
                img = img.permute(1, 2, 0).cpu().numpy()

                ax.imshow(img)
                ax.set_title(f"Pred: {pred_label}", fontsize=10)
                ax.axis("off")

    plt.tight_layout()

    # Log Grid to WandB
    wandb.log({"Test Predictions": wandb.Image(fig, caption="Per-Class Predictions (10x3)")})
    plt.close(fig)


# Call the function after testing
test_dataloader = data_module.test_dataloader()
log_test_predictions(best_model, test_dataloader)

wandb.finish()


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.39100000262260437    │
│         test_loss         │     1.770098090171814     │
└───────────────────────────┴───────────────────────────┘

Test Accuracy,▁
epoch,▁▁▁▂▂▂▂▂▂▂▃▃▃▃▄▄▄▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇█
test_acc,▁
test_loss,▁
train_acc,▁▂▂▃▄▄▃▄▃▅▃▃▄▄▆▅▆▆▆▅▅█▇▆█
train_loss,█▇▇▇▅▆▅▅▆▅▇▄▅▄▃▃▄▂▂▃▂▁▁▃▁
trainer/global_step,▁▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇▇▇████
val_acc,▁▂▅▆▇▆▇██▆
val_loss,██▅▄▃▃▂▁▃▄
Test Accuracy,0.391
epoch,10


# PART B

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import pytorch_lightning as pl
import wandb
from pytorch_lightning.loggers import WandbLogger
from torchvision.datasets import ImageFolder
from kaggle_secrets import UserSecretsClient

# Load WandB API Key
user_secrets = UserSecretsClient()
wandb_key = user_secrets.get_secret("wandb")
wandb.login(key=wandb_key)

# Define Sweep Configuration
sweep_config = {
    "method": "grid",
    "metric": {"name": "val_acc", "goal": "maximize"},
    "parameters": {
        "epochs": {"values": [5]},
        "batch_size": {"values": [64, 128]},
        "denselayer_size": {"values": [64, 128]},
        "l_rate": {"values": [0.001, 0.0001]},
        "optimizer": {"values": ["Adam"]},
        "dropout": {"values": [0.2, 0.4]},
        "model_version": {"values": ["resnet50"]},
        "activation": {"values": ["relu", "leakyrelu"]}
    }
}

# Create Sweep
sweep_id = wandb.sweep(sweep_config, project="Pretrained_Resnet50-Model")

# Define Data Transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

class FineTuneModel(pl.LightningModule):
    def __init__(self, num_classes=10, model_version="resnet50", denselayer_size=128, dropout=0.4, l_rate=0.001, activation="relu"):
        super(FineTuneModel, self).__init__()
        self.learning_rate = l_rate
        self.activation_fn = nn.ReLU() if activation == "relu" else nn.LeakyReLU()
        
        self.model = models.__dict__[model_version](pretrained=True)

        # Freeze all layers initially
        for param in self.model.parameters():
            param.requires_grad = False

        # Modify the final classification layer
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Sequential(
            nn.Linear(num_ftrs, denselayer_size),
            self.activation_fn,
            nn.Dropout(dropout),
            nn.Linear(denselayer_size, num_classes)
        )

        self.criterion = nn.CrossEntropyLoss()

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        outputs = self(x)
        loss = self.criterion(outputs, y)
        acc = (outputs.argmax(dim=1) == y).float().mean()
        self.log("train_loss", loss, prog_bar=True)
        self.log("train_acc", acc, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        outputs = self(x)
        loss = self.criterion(outputs, y)
        acc = (outputs.argmax(dim=1) == y).float().mean()
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", acc, prog_bar=True)

    def test_step(self, batch, batch_idx):
        x, y = batch
        outputs = self(x)
        loss = self.criterion(outputs, y)
        acc = (outputs.argmax(dim=1) == y).float().mean()
        self.log("test_loss", loss, prog_bar=True)
        self.log("test_acc", acc, prog_bar=True)
        return loss

    def configure_optimizers(self):
        optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate)
        return optimizer

# Training Function for WandB Sweeps
def train():
    wandb.init()

    # Fetch hyperparameters
    config = wandb.config
    batch_size = config.batch_size
    epochs = config.epochs
    model_version = config.model_version
    denselayer_size = config.denselayer_size
    dropout = config.dropout
    l_rate = config.l_rate
    activation = config.activation

    # Load Data
    data_module = INaturalistDataModule(
        data_dir='/kaggle/input/inaturalist-dataset/inaturalist_12K',
        batch_size=batch_size
    )
    data_module.setup()
    train_loader = data_module.train_dataloader()
    val_loader = data_module.val_dataloader()
    test_loader = data_module.test_dataloader()

    # Initialize Model
    model = FineTuneModel(num_classes=10, model_version=model_version, denselayer_size=denselayer_size, dropout=dropout,
                          l_rate=l_rate, activation=activation)

    # Set up WandB Logger
    wandb_logger = WandbLogger(project="Pretrained_Resnet50-Model")

    # Train Model
    trainer = pl.Trainer(
        max_epochs=epochs,
        logger=wandb_logger,
        accelerator="gpu" if torch.cuda.is_available() else "cpu"
    )
    trainer.fit(model, train_loader, val_loader)

    # Test Best Model
    trainer.test(model, test_loader)

# Run the Sweep Agent
wandb.agent(sweep_id, function=train, count=20)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: da24m026 (da24m026-indian-institute-of-technology-madras). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: x2em4v3x
Sweep URL: https://wandb.ai/da24m026-indian-institute-of-technology-madras/Pretrained_Resnet50-Model/sweeps/x2em4v3x


wandb: Agent Starting Run: xoh7wp0s with config:
wandb: 	activation: relu
wandb: 	batch_size: 64
wandb: 	denselayer_size: 64
wandb: 	dropout: 0.2
wandb: 	epochs: 5
wandb: 	l_rate: 0.001
wandb: 	model_version: resnet50
wandb: 	optimizer: Adam


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 179MB/s] 
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loggers/wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If t

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

wandb: Agent Starting Run: 6z3cdpiz with config:
wandb: 	activation: leakyrelu
wandb: 	batch_size: 64
wandb: 	denselayer_size: 64
wandb: 	dropout: 0.4
wandb: 	epochs: 5
wandb: 	l_rate: 0.0001
wandb: 	model_version: resnet50
wandb: 	optimizer: Adam


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]